In [1]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# set up resolution
resolution = 10 # pixels/um

# Define cell size
sx = 16
sy = 32
cell = mp.Vector3(sx,sy,0)

# Define PML
dpml = 1.0
pml_layers = [mp.PML(dpml)]

In [5]:
# define padding
pad = 4 # Padding between waveguide and edge
w = 1 # Waveguide width

# define waveguide centers
wvg_xcen = 0.5*(sx -w - 2*pad)
wvg_ycen = 0.5*(sy -w - 2*pad)

# define geometry
geometry = [mp.Block(size = mp.Vector3(mp.inf, w, mp.inf),
                     center = mp.Vector3(0,wvg_ycen,0),
                     material = mp.Medium(epsilon = 12))]

In [8]:
# define sources
fcen = 0.15
df = 0.1 # pulse width
sources = [mp.Source(mp.GaussianSource(fcen,fwidth=df),
                     component=mp.Ez,
                     center = mp.Vector3(-0.5*sx+dpml, wvg_ycen ,0),
                     size = mp.Vector3(0,w,0))]

In [9]:
# define simulation
sim = mp.Simulation(cell_size=cell,
                    resolution=resolution,
                    geometry=geometry,
                    sources=sources,
                    boundary_layers=pml_layers)

nfreq = 100 # number of frequencies to compute flux

# reflected flux
refl_fr = mp.FluxRegion(center=mp.Vector3(-0.5*sx+dpml + 0.5, wvg_ycen,0),
                        size = mp.Vector3(0,2*w,0))
refl = sim.add_flux(fcen, df, nfreq, refl_fr)

# transmitted flux
tran_fr = mp.FluxRegion(center = mp.Vector3(0.5*sx - dpml, wvg_ycen, 0), size = mp.Vector3(0, 2*w,0))
tran = sim.add_flux(fcen, df, nfreq, tran_fr)